#### Illustration of the convolution of normal and logit-normal distributions
##### Accompanyment to "Modeling apparent Pb loss in zircon U-Pb geochronology", submitted to Geochronology (gchron-2023-6)
##### Revision 1 (R1) August 2023
By: Glenn R. Sharman, Department of Geosciences, University of Arkansas

In [ ]:
# Import required libraries
from scipy.stats import norm
from scipy.signal import convolve

import convFuncs as convFunc

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # For allowing preservation of fonts upon importing into Adobe Illustrator
matplotlib.rcParams['ps.fonttype'] = 42
import numpy as np

In [ ]:
# Define the parameters of the normal and logit-normal distributions

mu = 0 # Mean of normal distribution, in %
omega = 1.5 # Standard deviation of the normal distribution, in %

m = -4.5 # mu of logit-normal distribution
s = 1 # sigma of logit-normal distribution

n = 2000 # Number of analyses drawn from each distribution

xdif = 0.01 # X-axis discretization interval
x1 = -100 # Lower x-axis for plotting, in %
x2 = 100 # Upper x-axis for plotting, in %
x = np.arange(x1, x2+xdif, xdif)

In [ ]:
# Define the Gaussian distribution pdf and sample it randomly
rv_norm = norm(loc = mu, scale = omega)
norm_pdf = rv_norm.pdf(x)
norm_pdf = norm_pdf/np.sum(norm_pdf) # Normalize so area under the curve = 1
norm_n = rv_norm.rvs(size=n) # Draw randomly from the normal distribution

In [ ]:
# Define the logit-normal distribution and sample it randomly
rv_logitnorm = convFunc.logitnorm_gen(a=0.0, b=1.0)
logitnorm_pdf = rv_logitnorm.pdf(x=-x/100, m=m, s=s)
logitnorm_pdf[np.isnan(logitnorm_pdf)] = 0 # To avoid nan values (e.g., logit-norm distribution at values of 0 and 1)
logitnorm_pdf = logitnorm_pdf/np.sum(logitnorm_pdf) # Normalize so area under the curve = 1
logitnorm_n = -rv_logitnorm.rvs(m=m, s=s, size=n)*100

In [ ]:
# Add the two random, independent variables (Gaussian and logit-normal)
conv_n = norm_n + logitnorm_n

In [ ]:
# Convolve the normal and logit-normal distributions
conv_pdf = convolve(logitnorm_pdf, norm_pdf, mode='same')
conv_pdf = conv_pdf/np.sum(conv_pdf)

In [ ]:
# Create a figure to illustrate the results
fig, axs = plt.subplots(1, 3, figsize=(8, 2))

# X-axis plotting limits
x_1 = -10
x_2 = 10

bins = np.arange(x_1, x_2+0.5, 0.5)

axs[0].plot(x, norm_pdf, color='black')
axs[0].set_ylim(0,)
axs[0].set_xlim(x_1, x_2)
axs[0].set_yticks([])
ax0_hist = axs[0].twinx()
ax0_hist.hist(norm_n, alpha=0.5, bins=bins, color='gray')
axs[0].set_xlabel('Pb*/U offset (%)')

axs[1].plot(x, logitnorm_pdf, color='black')
axs[1].set_ylim(0,)
axs[1].set_xlim(x_1, 0)
axs[1].set_yticks([])
ax1_hist = axs[1].twinx()
ax1_hist.hist(logitnorm_n, alpha=0.5, bins=bins, color='gray')
axs[1].set_xlabel('Pb*/U offset (%)')

axs[2].plot(x, conv_pdf, color='black')
axs[2].set_ylim(0,)
axs[2].set_xlim(x_1, x_2)
axs[2].set_yticks([])
ax2_hist = axs[2].twinx()
ax2_hist.hist(conv_n, alpha=0.5, bins=bins, color='gray');
axs[2].set_xlabel('Pb*/U offset (%)')

plt.tight_layout()